In [1]:
# Import PuLP modeler functions
from pulp import *

In [2]:
# Creates a list of all the supply nodes
Plants = ["San Francisco",
          "Los Angeles",
          "Phoenix",
          "Denver"]

# Creates a dictionary of lists for the number of units of supply and fixed cost 
supplyData = {#Plant     Supply  Fixed Cost
          "San Francisco":[1700, 70000],
          "Los Angeles"  :[2000, 70000],
          "Phoenix"      :[1700, 65000],
          "Denver"       :[2000, 70000]
          }

# Creates a list of all demand nodes
Stores = ["San Diego",
          "Barstow",
          "Tucson",
          "Dallas"]

# Creates a dictionary for the number of units of demand and store
demand = { #Store    Demand
          "San Diego":1700,
          "Barstow"  :1000,
          "Tucson"   :1500,
          "Dallas"   :1200
          }

# Creates a list of costs for each transportation path in matrix format
costs = [  #Stores
         #SD BA TU DA
         [5, 3, 2, 6], #SF
         [4, 7, 8, 10],#LA    Plants
         [6, 5, 3, 8], #PH
         [9, 8, 6, 5]  #DE         
         ]

In [3]:
# Creates a list of tuples containing transporting routes
Routes = [(p,s) for p in Plants for s in Stores]


In [4]:
# Splits the dictionaries for supply data
(supply,fixedCost) = splitDict(supplyData)

In [5]:
# The cost data is made into a dictionary
costs = makeDict([Plants,Stores],costs,0)

In [6]:
# Creates the problem variables of the Flow 
flow = LpVariable.dicts("Route",(Plants,Stores),0,None,LpInteger)

In [7]:
# Creates the 'build' variables of whether to build the Plants or not
build = LpVariable.dicts("BuildaPlant",Plants,0,1,LpInteger)

In [8]:
# Creates the 'prob' variable to contain the problem data, reads the data set
prob = LpProblem("Computer Plant Problem",LpMinimize)

In [9]:
# The objective function is added to prob - The sum of the transportation costs and the building fixed costs
#prob=transportation cost+fixed cost
prob += lpSum([flow[p][s]*costs[p][s] for (p,s) in Routes])+lpSum([fixedCost[p]*build[p] for p in Plants]),"Total Costs"

#the constraints
# The Supply maximum constraints are added for each supply node (plant)
for p in Plants:
    prob += lpSum([flow[p][s] for s in Stores])<=supply[p]*build[p], "Sum of Products out of Plant %s"%p

# The Demand minimum constraints are added for each demand node (store)
for s in Stores:
    prob += lpSum([flow[p][s] for p in Plants])>=demand[s], "Sum of Products into Stores %s"%s


In [10]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

1

In [11]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])


Status: Optimal


In [12]:
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

BuildaPlant_Denver = 0.0
BuildaPlant_Los_Angeles = 1.0
BuildaPlant_Phoenix = 1.0
BuildaPlant_San_Francisco = 1.0
Route_Denver_Barstow = 0.0
Route_Denver_Dallas = 0.0
Route_Denver_San_Diego = 0.0
Route_Denver_Tucson = 0.0
Route_Los_Angeles_Barstow = 0.0
Route_Los_Angeles_Dallas = 300.0
Route_Los_Angeles_San_Diego = 1700.0
Route_Los_Angeles_Tucson = 0.0
Route_Phoenix_Barstow = 0.0
Route_Phoenix_Dallas = 200.0
Route_Phoenix_San_Diego = 0.0
Route_Phoenix_Tucson = 1500.0
Route_San_Francisco_Barstow = 1000.0
Route_San_Francisco_Dallas = 700.0
Route_San_Francisco_San_Diego = 0.0
Route_San_Francisco_Tucson = 0.0


In [13]:
# The optimised objective function value is printed to the screen    
print("Total Costs = ", value(prob.objective))

Total Costs =  228100.0
